In [14]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from scipy import stats
import ipy_table as tbl

run_id = 359
qid = 1603

In [3]:
topics = Topic.objects.filter(run_id=359)
dts = DocTopic.objects.filter(run_id=359)

p1total = dts.filter(doc__PY__gt=2009,doc__PY__lt=2014).aggregate(t=Sum('score'))
p2total = dts.filter(doc__PY__gt=2013,doc__PY__lt=2018).aggregate(t=Sum('score'))

In [3]:
p1total

{'t': 313.828131828615}

In [4]:
p2total

{'t': 482.821877065289}

In [4]:
topic_objects = []
for t in topics:
    t1 = dts.filter(doc__PY__gt=2007,doc__PY__lt=2013,topic=t).aggregate(t=Sum('score'))
    t2 = dts.filter(doc__PY__gt=2012,doc__PY__lt=2018,topic=t).aggregate(t=Sum('score'))
    p1 = t1['t'] / p1total['t']
    p2 = t2['t'] / p2total['t']
    growth = (p2 - p1) / p1*100
    t.growth = float(growth)
    t.save()
    print(t.title)
    print(t.growth)
    t.save()
    topic_objects.append(t)
    
print(topic_objects)
for top in topic_objects:
    print(top.growth)
    print(top.run_id_id)
    top.save()
    
#Topic.objects.update(topics)
t.save()

System design
23.176104541759575
Travel behaviour
26.658320187443852
Thermal comfort
39.80222542341183
e-Vehicles
55.168655806927866
Waste water
1.6213933630598087
Rail transit
31.969330862118177
Air quality
8.518597136711016
Climate change mitigation
19.53111477394588
Heating & cooling systems
22.89792998007299
Transport services & provisioning
31.042651932087455
Car parking
54.73805366168138
Bus rapid transit
21.779998006709835
Road tolls
-24.83915943557506
Planning & governance
23.89914075879715
Active travel
25.422888449215947
Network optimisation
32.51453418324543
Energy efficiency & consumption
26.01732271264118
Building energy standards
44.566543856177624
Urban form
58.8256839296722
GHG emissions
29.087530478876317
Waste management
-19.22734291650614
Public transport
6.436837768603384
Congestion charging
-13.603061151579542
Household recycling
21.68757112005821
Charging stations
55.24653855042791
Building water use
52.224417531298265
Cycling
34.56100986115314
[<Topic: System des

In [25]:
d = 5
df = pd.DataFrame.from_dict(list(topics.values('title')))
for y in range(2000,2017):
    y1 = y-1
    y2 = y+1
    ry1 = y1-5
    ry2 = y2-5
    tp = "{} - {}".format(y1,y2)
    print(tp)
    for t in topics:
        t1 = dts.filter(doc__PY__gte=ry1,doc__PY__lte=ry2,topic=t).aggregate(t=Sum('score'))
        t2 = dts.filter(doc__PY__gte=y1,doc__PY__lte=y2,topic=t).aggregate(t=Sum('score'))
        growth = (t2['t'] - t1['t']) / t1['t']*100
        df.loc[df['title']==t.title,tp] = growth

df.head()
    


1999 - 2001
2000 - 2002
2001 - 2003
2002 - 2004
2003 - 2005
2004 - 2006
2005 - 2007
2006 - 2008
2007 - 2009
2008 - 2010
2009 - 2011
2010 - 2012
2011 - 2013
2012 - 2014
2013 - 2015
2014 - 2016
2015 - 2017


,title,1999 - 2001,2000 - 2002,2001 - 2003,2002 - 2004,2003 - 2005,2004 - 2006,2005 - 2007,2006 - 2008,2007 - 2009,2008 - 2010,2009 - 2011,2010 - 2012,2011 - 2013,2012 - 2014,2013 - 2015,2014 - 2016,2015 - 2017
0,System design,43.962728,55.110646,56.697891,60.477865,63.315278,116.028552,116.297219,133.298177,135.966654,147.506642,143.147302,146.198253,162.160048,166.587891,149.887451,130.149765,91.869220
1,Travel behaviour,117.460072,134.563857,87.890667,152.090981,106.111343,97.818665,44.415912,51.610467,58.911412,76.193129,113.737740,147.341120,174.757217,163.267639,168.486475,130.579632,98.610191
2,Thermal comfort,97.641845,94.831036,78.290026,30.481156,48.891409,68.718794,122.987234,193.488302,337.247928,328.350092,269.667282,228.158596,199.400831,227.710304,183.393546,181.687225,110.064808
3,e-Vehicles,35.216661,33.486232,71.027415,46.362257,52.161954,58.436166,68.615830,95.602024,107.203749,151.168689,178.006321,201.502305,237.735766,241.481864,218.129908,181.360191,136.715409
4,Waste water,-3.702500,8.788139,22.317308,62.720546,75.986421,99.146613,115.153543,148.899793,116.044758,109.559433,65.787193,85.047374,59.558808,77.242168,64.813399,98.051968,83.225272


In [26]:
df.to_excel('../data/5year_growth.xlsx')

In [21]:
d = 5
df = pd.DataFrame.from_dict(list(topics.values('title')))

df.loc[df['title']=="System design","bla"] = "x"
df.head()

,title,bla
0,System design,x
1,Travel behaviour,NaN
2,Thermal comfort,NaN
3,e-Vehicles,NaN
4,Waste water,NaN


In [8]:
topic_table = topics.values('title','growth')
df = pd.DataFrame.from_dict(list(topic_table))
df.to_csv('../data/topic_growth_2010.csv')
df.sort_values('growth',ascending=False)

,growth,title
18,58.825684,Urban form
24,55.246539,Charging stations
3,55.168656,e-Vehicles
10,54.738054,Car parking
25,52.224418,Building water use
17,44.566544,Building energy standards
2,39.802225,Thermal comfort
26,34.561010,Cycling
15,32.514534,Network optimisation
5,31.969331,Rail transit


In [11]:
topic_objects = []
for t in topics:
    t1 = dts.filter(doc__PY__gte=2009,doc__PY__lte=2011,topic=t).aggregate(t=Sum('score'))
    t2 = dts.filter(doc__PY__gte=2014,doc__PY__lte=2016,topic=t).aggregate(t=Sum('score'))
    p1 = t1['t'] / p1total['t']
    p2 = t2['t'] / p2total['t']
    growth = (p2 - p1) / p1*100
    t.growth = float(growth)
    t.save()
    #print(t.title)
    #print(t.growth)
    t.save()
    topic_objects.append(t)
    
    #print(topic_objects)
    for top in topic_objects:
        #print(top.growth)
        #print(top.run_id_id)
        top.save()

    #Topic.objects.update(topics)
    t.save()

topic_table = topics.values('title','growth')
df = pd.DataFrame.from_dict(list(topic_table))
df.to_csv('../data/topic_growth_2010.csv')
df.sort_values('growth',ascending=False)

,growth,title
26,88.626673,Cycling
10,77.172394,Car parking
24,76.780847,Charging stations
17,73.460131,Building energy standards
25,68.749668,Building water use
2,63.665536,Thermal comfort
3,63.475523,e-Vehicles
18,62.542551,Urban form
9,60.943163,Transport services & provisioning
11,54.935589,Bus rapid transit


In [13]:
topic_objects = []
for t in topics:
    t1 = dts.filter(doc__PY__gte=2011,doc__PY__lte=2013,topic=t).aggregate(t=Sum('score'))
    t2 = dts.filter(doc__PY__gte=2014,doc__PY__lte=2016,topic=t).aggregate(t=Sum('score'))
    p1 = t1['t'] / p1total['t']
    p2 = t2['t'] / p2total['t']
    growth = (p2 - p1) / p1*100
    t.growth = float(growth)
    t.save()
    #print(t.title)
    #print(t.growth)
    t.save()
    topic_objects.append(t)
    
    #print(topic_objects)
    for top in topic_objects:
        #print(top.growth)
        #print(top.run_id_id)
        top.save()

    #Topic.objects.update(topics)
    t.save()

topic_table = topics.values('title','growth')
df = pd.DataFrame.from_dict(list(topic_table))
df.to_csv('../data/topic_growth_2010.csv')
df.sort_values('growth',ascending=False)

,growth,title
24,17.805447,Charging stations
9,7.458628,Transport services & provisioning
2,6.205350,Thermal comfort
11,5.047322,Bus rapid transit
17,2.378440,Building energy standards
26,1.652696,Cycling
18,1.568342,Urban form
25,-0.100455,Building water use
10,-0.629261,Car parking
5,-2.644010,Rail transit
